In [1]:
from transformers import GPT2LMHeadModel, AutoTokenizer
import torch
from transformers import pipeline
device = torch.device("cuda")
import pandas as pd
import math
import plotly.express as px
tokenizer = AutoTokenizer.from_pretrained('/home/phil/Geraldene/transformers/examples/pytorch/language-modeling/ZymCTRL')
model = GPT2LMHeadModel.from_pretrained('/home/phil/Geraldene/transformers/examples/pytorch/language-modeling/ZymCTRL').to(device)

/home/phil/anaconda3/envs/omegafold/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def calculatePerplexity(input_ids,model,tokenizer):
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    loss, logits = outputs[:2]
    return math.exp(loss)

In [3]:
#load pickle file 
df = pd.read_pickle('unique_labels_eval_set.pkl')

In [6]:
import random
random_validation_labels = random.choices(list(df), k=10)

In [7]:
random_validation_labels

['2.1.1.n11',
 '1.6.99.2',
 '2.1.1.300',
 '2.6.1.110',
 '3.4.21.38',
 '1.3.98.4',
 '1.14.14.15',
 '5.4.99.50',
 '1.14.20.8',
 '5.3.1.n1']

In [8]:
# 1. Run directly from the HuggingFace pipeline
ppls = {}
#model = pipeline('text-generation', model="nferruz/ZymCTRL")
for i in random_validation_labels:
    sequences = tokenizer.encode(i, return_tensors='pt').to(device)
    outputs = model.generate(
        sequences,
        top_k=9,
        repetition_penalty=1.2,
        max_length=1024,
        eos_token_id=1,
        pad_token_id=0,
        do_sample=True,
        num_return_sequences=20)
    ppls[i] = [calculatePerplexity(output, model, tokenizer) for output in outputs ]

In [12]:
import numpy as np
for label, ppl_value in ppls.items():
    print(label)
    print(np.mean(ppl_value))

2.1.1.n11
5.967945800314948
1.6.99.2
7.534536075026456
2.1.1.300
4.080559569391931
2.6.1.110
3.2126121771036447
3.4.21.38
7.553030041108123
1.3.98.4
4.388164780701416
1.14.14.15
2.689296617615733
5.4.99.50
3.728291451907218
1.14.20.8
3.140201889914207
5.3.1.n1
2.5311253483490637
